### Load Simulated daily and simulated 5-minute data

In [17]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from arch import arch_model
import os

data_folder = "C:/Users/codyr/Desktop/Coding_/Quant_Trading_Basics"

daily_df = pd.read_csv(os.path.join(data_folder, "simulated_daily_data.csv"))

daily_df['Date'] = pd.to_datetime(daily_df['Date'])

daily_df = daily_df.set_index('Date')

daily_df['log_ret'] = np.log(daily_df['Adj Close']).diff()

daily_df = daily_df.drop('Unnamed: 7', axis=1)

intraday_5min_df = pd.read_csv(os.path.join(data_folder, 'simulated_5min_data.csv'))

intraday_5min_df['datetime'] = pd.to_datetime(intraday_5min_df['datetime'])

intraday_5min_df = intraday_5min_df.set_index('datetime')

intraday_5min_df['date'] = intraday_5min_df.index.date

intraday_5min_df = intraday_5min_df.drop('Unnamed: 6', axis=1)

intraday_5min_df.index.date

array([datetime.date(2021, 9, 29), datetime.date(2021, 9, 29),
       datetime.date(2021, 9, 29), ..., datetime.date(2023, 9, 20),
       datetime.date(2023, 9, 20), datetime.date(2023, 9, 20)],
      dtype=object)